In [36]:
import pandas as pd
import tensortrade.env.default as default

from tensortrade.data.cdd import CryptoDataDownload
from tensortrade.feed.core import Stream, DataFeed
from tensortrade.oms.exchanges import Exchange
from tensortrade.oms.services.execution.simulated import execute_order
from tensortrade.oms.instruments import USD, BTC, ETH
from tensortrade.oms.wallets import Wallet, Portfolio
from tensortrade.agents import DQNAgent as Agent


%matplotlib inline

In [37]:


data = pd.read_csv('C:/Users/Annoy/Desktop/Spyder/Useful Code Snippets/Data/Historic BTC-USD data/BTC-USD_Cleaned_Megafile_01_01_2020-12_31_2020.csv')
data.drop('Unnamed: 0', axis=1, inplace=True)

In [38]:
data.columns[1:6].join(data.columns[7:],how='outer')

Index(['close', 'high', 'low', 'open', 'rel_close', 'rel_high', 'rel_low',
       'rel_open', 'rel_volume', 'volume'],
      dtype='object')

In [39]:
def rsi(price: Stream[float], period: float) -> Stream[float]:
    r = price.diff()
    upside = r.clamp_min(0).abs()
    downside = r.clamp_max(0).abs()
    rs = upside.ewm(alpha=1 / period).mean() / downside.ewm(alpha=1 / period).mean()
    return 100*(1 - (1 + rs) ** -1)


def macd(price: Stream[float], fast: float, slow: float, signal: float) -> Stream[float]:
    fm = price.ewm(span=fast, adjust=False).mean()
    sm = price.ewm(span=slow, adjust=False).mean()
    md = fm - sm
    signal = md - md.ewm(span=signal, adjust=False).mean()
    return signal


features = []
for c in data.columns[7:]:
    s = Stream.source(list(data[c]), dtype="float").rename(data[c].name)
    features += [s]

cp = Stream.select(features, lambda s: s.name == "close")

features = [
    cp.log().diff().rename("lr"),
    rsi(cp, period=20).rename("rsi"),
    macd(cp, fast=10, slow=50, signal=5).rename("macd")
]
feed = DataFeed(features)
feed.compile()

In [40]:

for i in range(5):
    print(feed.next())

{'lr': nan, 'rsi': nan, 'macd': 0.0}
{'lr': 0.0003027565518642916, 'rsi': 100.0, 'macd': 0.20629827688632457}
{'lr': 0.00048115243209423397, 'rsi': 100.0, 'macd': 0.6262172951062692}
{'lr': -0.0001422296593069916, 'rsi': 83.6954567070356, 'macd': 0.6997067175705995}
{'lr': 0.0002830449802466717, 'rsi': 87.84667813121295, 'macd': 0.8743509966006069}


In [41]:
bitstamp = Exchange("bitstamp", service=execute_order)(
    Stream.source(list(data["close"]), dtype="float").rename("USD-BTC")
)

portfolio = Portfolio(USD, [
    Wallet(bitstamp, 10000 * USD),
    Wallet(bitstamp, 10 * BTC)
])


renderer_feed = DataFeed([
    Stream.source(list(data["Date"])).rename("date"),
    Stream.source(list(data["open"]), dtype="float").rename("open"),
    Stream.source(list(data["high"]), dtype="float").rename("high"),
    Stream.source(list(data["low"]), dtype="float").rename("low"),
    Stream.source(list(data["close"]), dtype="float").rename("close"), 
    Stream.source(list(data["volume"]), dtype="float").rename("volume") 
])


env = default.create(
    portfolio=portfolio,
    action_scheme="managed-risk",
    reward_scheme="risk-adjusted",
    feed=feed,
    renderer_feed=renderer_feed,
    renderer=default.renderers.PlotlyTradingChart(),
    window_size=20
)

In [42]:
env.observer.feed.next()

{'internal': {'bitstamp:/USD-BTC': 7166.39,
  'bitstamp:/USD:/free': 10000.0,
  'bitstamp:/USD:/locked': 0.0,
  'bitstamp:/USD:/total': 10000.0,
  'bitstamp:/BTC:/free': 10.0,
  'bitstamp:/BTC:/locked': 0.0,
  'bitstamp:/BTC:/total': 10.0,
  'bitstamp:/BTC:/worth': 71663.90000000001,
  'net_worth': 81663.90000000001},
 'external': {'lr': nan, 'rsi': nan, 'macd': 0.0},
 'renderer': {'date': '01_01_2020',
  'open': 7174.33,
  'high': 7174.33,
  'low': 7165.02,
  'close': 7166.39,
  'volume': 6.27832277}}

In [ ]:
agent = Agent(env)

agent.train(n_steps=200, n_episodes=100, save_path="agents/")

FigureWidget({
    'data': [{'close': array([7166.39, 7168.56, 7172.01, ..., 7202.8 , 7208.81, 7209.81]),
    …